In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
## Load dataset

df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
## Drop irrelevant features

df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Encode categorical features

label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])


In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [8]:
## One hot encoding geography column

from sklearn.preprocessing import OneHotEncoder

one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(df[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [10]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
## combine one hot enoded data with original dataset

df = pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
## save the encoders and scaler

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_got_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)


In [14]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
## divide the dataset into independent and dependent features
x = df.drop('Exited',axis=1)
y = df['Exited']

## train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [17]:
x_train.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [18]:
## scaling features

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

y_train = y_train.to_numpy()
y_test= y_test.to_numpy()

In [19]:
## saving scaler file

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN Implementation

In [20]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

In [21]:
x_train = torch.tensor(x_train)
x_test = torch.tensor(x_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.layer1 = nn.Linear(in_features=x_train.shape[1],out_features=64)
        self.layer2 = nn.Linear(in_features=64,out_features=32)
        self.layer3 = nn.Linear(in_features=32,out_features=1)
    
    def forward(self,x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        x = torch.relu(x)
        x = self.layer3(x)
        x = torch.sigmoid(x)

        return x
        


In [23]:
class EarlyStopping:
    def __init__(self,patience = 10,verbose=False,delta=0.05):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.delta = delta
        self.best_loss = None
        self.best_model = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = model.state_dict()
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.best_model = model.state_dict()
            self.counter = 0


early_stopping = EarlyStopping(patience=10,verbose=True)

In [24]:
network = Net()

In [25]:
summary(network,input_size=(64,x_train.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 64, 64]             832
            Linear-2               [-1, 64, 32]           2,080
            Linear-3                [-1, 64, 1]              33
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.01
Estimated Total Size (MB): 0.06
----------------------------------------------------------------


In [26]:
optimizer = optim.Adam(network.parameters(),lr=0.01)
criterion = nn.BCELoss()

In [27]:
from torch.utils.data import DataLoader,TensorDataset

train_data = TensorDataset(x_train,y_train)
test_data = TensorDataset(x_test,y_test)

train_loader  = DataLoader(train_data,shuffle=True)
val_loader  = DataLoader(test_data,shuffle=True)

In [28]:
import datetime

log_dir = "logs/fit_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(log_dir=log_dir)

In [29]:
def get_num_correct(preds,targets):
    return preds.argmax(dim=1).eq(targets).sum().item()

def train(epoch):
    running_loss = 0.0  # For accumulating the loss value
    correct = 0
    total = 0

    network.train()

    for input, target in train_loader:
        input = input.float()  # If your input needs an extra dimension
        target = target.unsqueeze(1).float()
        optimizer.zero_grad()
        preds = network(input)
        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()  # Accumulate scalar loss value
        correct += get_num_correct(preds, target)
        total += target.size(0)
    
    avg_loss = running_loss / total
    accuracy = correct / total

    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Correct: {correct}, Accuracy: {accuracy:.4f}')

    # Log the loss and correct count to TensorBoard
    writer.add_scalar('Loss/train', avg_loss, epoch)
    writer.add_scalar('Correct/train', correct, epoch)
    writer.add_scalar('Accuracy/train', accuracy, epoch)


def test(epoch):
    network.eval()

    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  # Disable gradient calculation for validation/testing
        for input, target in val_loader:
            input = input.float()
            target = target.unsqueeze(1).float()
            preds = network(input)
            loss = criterion(preds, target)

            val_loss += loss.item()
            val_correct += get_num_correct(preds, target)
            val_total += target.size(0)

    avg_val_loss = val_loss / val_total
    val_accuracy = val_correct / val_total

    print(f'Epoch: {epoch}, Validation Loss: {avg_val_loss:.4f}, Correct: {val_correct}, Accuracy: {val_accuracy:.4f}')

    # Log the validation metrics
    writer.add_scalar('Loss/val', avg_val_loss, epoch)
    writer.add_scalar('Correct/val', val_correct, epoch)
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)

    # Early stopping
    early_stopping(avg_val_loss, network)
    if early_stopping.early_stop:
        print("Early stopping triggered.")
        network.load_state_dict(early_stopping.best_model)  # Load the best model's weights


# Main training loop
for epoch in range(100):
    train(epoch)
    test(epoch)
    if early_stopping.early_stop:
        break  # Stop training if early stopping is triggered

# Close TensorBoard writer after training is finished
writer.close()

Epoch: 0, Train Loss: 0.4160, Correct: 6356, Accuracy: 0.7945
Epoch: 0, Validation Loss: 0.3588, Correct: 1607, Accuracy: 0.8035
Epoch: 1, Train Loss: 0.3900, Correct: 6356, Accuracy: 0.7945
Epoch: 1, Validation Loss: 0.3936, Correct: 1607, Accuracy: 0.8035
EarlyStopping counter: 1 out of 10
Epoch: 2, Train Loss: 0.3812, Correct: 6356, Accuracy: 0.7945
Epoch: 2, Validation Loss: 0.3553, Correct: 1607, Accuracy: 0.8035
EarlyStopping counter: 2 out of 10
Epoch: 3, Train Loss: 0.3796, Correct: 6356, Accuracy: 0.7945
Epoch: 3, Validation Loss: 0.3581, Correct: 1607, Accuracy: 0.8035
EarlyStopping counter: 3 out of 10
Epoch: 4, Train Loss: 0.3691, Correct: 6356, Accuracy: 0.7945
Epoch: 4, Validation Loss: 0.3933, Correct: 1607, Accuracy: 0.8035
EarlyStopping counter: 4 out of 10
Epoch: 5, Train Loss: 0.3764, Correct: 6356, Accuracy: 0.7945
Epoch: 5, Validation Loss: 0.3694, Correct: 1607, Accuracy: 0.8035
EarlyStopping counter: 5 out of 10
Epoch: 6, Train Loss: 0.3685, Correct: 6356, Accura

In [30]:
model_script = torch.jit.script(network)

# Save the TorchScript model
model_script.save('model_script.pt')
print("Model saved successfully!")

Model saved successfully!
